# Exercício 2



## Enunciado

Para a viga em balanço da figura abaixo tratada como um problema de Estado Plano de Tensões determine a matriz de rigidez e o vetor de forças para as seguintes discretizações:

a) Com 2 elementos triangulares lineares conforme mostrado nesta figura.

b) Com 1 elemento retangular com os nós 1,2,3 e 4.

Determine os deslocamentos livres considerando os nós 3 e 4 presos(deslocamentos nulos). 

Determine também os valores das tensões, segundo o sistema de eixos indicado nos pontos nodais e nos pontos centrais dos elementos considerados. Considere o carregamento indicado nesta figura com a distribuição de carga onde p = 150*(N+1)/20 . Adotar E (200.000) e Poisson(0.3) indicados na figura. Atente para trabalhar com unidades coerentes e que a espessura também está dada (t = 1). Utilize para as medidas do domínio o valor a = b = 4 + (N-1)/20.


![](ex2.png)

## Resolução

### Substituindo N

Temos que $N = 1$, logo as dimensões do domínio serão:

$$
a = b = 4
$$

### Coordenadas dos nós

* Nó 1: $(4,4)$
* Nó 2: $(4,0)$
* Nó 3: $(0,4)$
* Nó 4: $(0,0)$

### Nós dos elementos

* Elemento 1: (1, 2, 3)
* Elemento 2: (2, 3, 4)


### a) Elementos Triangulares

#### Funções de Forma

Claro, vamos detalhar melhor os passos 2 e 3 para a formulação da matriz de rigidez de elementos triangulares em estado plano de tensões, focando nas funções de forma e nas relações de deformação e tensão.

Para um elemento triangular com três nós (nós 1, 2, e 3), as funções de forma lineares $ N_1 $, $ N_2 $, e $ N_3 $ podem ser definidas em termos das coordenadas dos nós. Sejam $ (x_i, y_i) $ as coordenadas dos nós $ i = 1, 2, 3 $, as funções de forma são:

$$ N_i = a_i + b_i x + c_i y $$

Onde $ a_i $, $ b_i $, e $ c_i $ são constantes determinadas pela geometria do elemento e podem ser calculadas usando a fórmula:

$$ a_i = \frac{x_j y_k - x_k y_j}{2A}, \quad b_i = \frac{y_j - y_k}{2A}, \quad c_i = \frac{x_k - x_j}{2A} $$

Aqui, $ (j, k) $ é um par de índices que são diferentes de $ i $, e $ A $ é a área do elemento triangular.

#### Deformação e Tensão

A deformação em um elemento plano pode ser expressa em termos dos deslocamentos nodais. Para um elemento triangular, os deslocamentos nodais são $ u_1, v_1, u_2, v_2, u_3, v_3 $, onde $ u $ e $ v $ são os deslocamentos nos eixos $ x $ e $ y $, respectivamente. A matriz de deformação $ [B] $ relaciona a deformação com os deslocamentos nodais:

$$
[B] = \begin{bmatrix}
\frac{\partial N_1}{\partial x} & 0 & \frac{\partial N_2}{\partial x} & 0 & \frac{\partial N_3}{\partial x} & 0 \\
0 & \frac{\partial N_1}{\partial y} & 0 & \frac{\partial N_2}{\partial y} & 0 & \frac{\partial N_3}{\partial y} \\
\frac{\partial N_1}{\partial y} & \frac{\partial N_1}{\partial x} & \frac{\partial N_2}{\partial y} & \frac{\partial N_2}{\partial x} & \frac{\partial N_3}{\partial y} & \frac{\partial N_3}{\partial x}
\end{bmatrix}
$$

As tensões são relacionadas às deformações pela lei de Hooke para materiais isotrópicos e homogêneos. A matriz de elasticidade $ [D] $ para estado plano de tensões é:

$$
[D] = \frac{E}{(1-\nu^2)} \begin{bmatrix}
1 & \nu & 0 \\
\nu & 1 & 0 \\
0 & 0 & \frac{1-\nu}{2}
\end{bmatrix}
$$

Onde $ E $ é o módulo de elasticidade e $ \nu $ é o coeficiente de Poisson.

A relação entre tensão e deformação é dada por:

$$
[\sigma] = [D][\epsilon]
$$

Onde $ [\sigma] $ é a matriz de tensão e $ [\epsilon] $ é a matriz de deformação.

Combinando essas relações, podemos calcular a matriz de rigidez local para o elemento. É importante notar que, na prática, a aplicação destas fórmulas e a integração necessária para calcular a matriz de rigidez podem exigir um conhecimento detalhado de métodos numéricos, especialmente para elementos de formas mais complexas ou materiais não homogêneos.

#### Montagem da Matriz de Rigidez


A matriz de rigidez $K$ de um elemento triangular em estado plano de tensões é expressa por:

$$
[K] = \int _A [B]^T[D][B]dA
$$

$$
K = tAB^T DB
$$

onde:

* $t$ é a espessura do elemento,
* $A$ é a área do elemento triangular,
* $B$ é a matriz de gradiente de deslocamento
* $D$ é a matriz de elasticidade plana.

A matriz de elasticidade $D$ é dada por:

$$
D = \dfrac{E}{1 - \nu^2}
\begin{bmatrix}
1 & \nu & 0 \\
\nu & 1 & 0 \\
0 & 0 & \dfrac{1 - \nu}{2}
\end{bmatrix}
$$

A matriz de deformação $B$ para um elemento triangular é derivada das coordenadas dos nós do elemento e é dada por:

$$
B = \dfrac{1}{2A}
\begin{bmatrix}
b_1 & 0 & b_2 & 0 & b_3 & 0 \\
0 & c_1 & 0 & c_2 & 0 & c_3 \\
c_1 & b_1 & c_2 & b_2 & c_3 & b_3
\end{bmatrix}
$$

onde $b_i$ e $c_i$ são as coordedanas do nó $i$ do elemento.

In [23]:
import numpy as np
import sympy as sp

# Propriedades do material
E = 200000  # Módulo de elasticidade (N/mm²)
nu = 0.3    # Coeficiente de Poisson
t = 1       # Espessura da viga (mm)

# Dimensões da viga
a = b = 4   # Dimensões para N = 1

# Carga aplicada
p = 150 * (1 + 1) / 20

# Coordenadas dos nós (em mm)
nodes = np.array([
    [a, b],  # Nó 1
    [a, 0],  # Nó 2
    [0, b],  # Nó 3
    [0, 0],  # Nó 4
])

# Elementos (triângulos) definidos pelos nós
elements = np.array([
    [1, 2, 3],  # Elemento 1
    [2, 3, 4]   # Elemento 2
])

# Número de nós e elementos
num_nodes = len(nodes)
num_elements = len(elements)

# Função para calcular a matriz de rigidez de um elemento triangular linear
def stiffness_matrix_triangle_linear(element, nodes, E, nu, t):
    # Coordenadas dos nós do elemento
    x = nodes[element - 1, 0]
    y = nodes[element - 1, 1]

    # Área do elemento
    A = 0.5 * np.linalg.det(np.array([[1, x[0], y[0]],
                                      [1, x[1], y[1]],
                                      [1, x[2], y[2]]]))

    # Matriz B
    B = 1 / (2 * A) * np.array([
        [y[1] - y[2], 0, y[2] - y[0], 0, y[0] - y[1], 0],
        [0, x[2] - x[1], 0, x[0] - x[2], 0, x[1] - x[0]],
        [x[2] - x[1], y[1] - y[2], x[0] - x[2], y[2] - y[0], x[1] - x[0], y[0] - y[1]]
    ])

    print('Matriz B do elemento {}:'.format(element))
    display(sp.Matrix(B))

    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    print('Matriz D do elemento {}:'.format(element))
    display(sp.Matrix(D))

    # Matriz de rigidez do elemento
    k = t * A * np.dot(np.dot(B.T, D), B)

    return k


In [24]:
k1 = stiffness_matrix_triangle_linear(elements[0], nodes, E, nu, t)
print('Matriz de Rigidez para o elemento 1:')
display(sp.Matrix(k1).evalf(3))

Matriz B do elemento [1 2 3]:


Matrix([
[0.25,    0,     0,     0, -0.25,     0],
[   0, 0.25,     0, -0.25,     0,     0],
[0.25, 0.25, -0.25,     0,     0, -0.25]])

Matriz D do elemento [1 2 3]:


Matrix([
[ 219780.21978022, 65934.0659340659,                0],
[65934.0659340659,  219780.21978022,                0],
[               0,                0, 76923.0769230769]])

Matriz de Rigidez para o elemento 1:


Matrix([
[-1.48e+5, -7.14e+4,  3.85e+4,  3.3e+4,  1.1e+5,  3.85e+4],
[-7.14e+4, -1.48e+5,  3.85e+4,  1.1e+5,  3.3e+4,  3.85e+4],
[ 3.85e+4,  3.85e+4, -3.85e+4,       0,       0, -3.85e+4],
[  3.3e+4,   1.1e+5,        0, -1.1e+5, -3.3e+4,        0],
[  1.1e+5,   3.3e+4,        0, -3.3e+4, -1.1e+5,        0],
[ 3.85e+4,  3.85e+4, -3.85e+4,       0,       0, -3.85e+4]])

In [25]:
k2 = stiffness_matrix_triangle_linear(elements[1], nodes, E, nu, t)
print('Matriz de Rigidez para o elemento 2:')
display(sp.Matrix(k2).evalf(3))

Matriz B do elemento [2 3 4]:


Matrix([
[0.25,    0,    0,    0, -0.25,     0],
[   0,    0,    0, 0.25,     0, -0.25],
[   0, 0.25, 0.25,    0, -0.25, -0.25]])

Matriz D do elemento [2 3 4]:


Matrix([
[ 219780.21978022, 65934.0659340659,                0],
[65934.0659340659,  219780.21978022,                0],
[               0,                0, 76923.0769230769]])

Matriz de Rigidez para o elemento 2:


Matrix([
[ 1.1e+5,        0,        0,  3.3e+4,  -1.1e+5,  -3.3e+4],
[      0,  3.85e+4,  3.85e+4,       0, -3.85e+4, -3.85e+4],
[      0,  3.85e+4,  3.85e+4,       0, -3.85e+4, -3.85e+4],
[ 3.3e+4,        0,        0,  1.1e+5,  -3.3e+4,  -1.1e+5],
[-1.1e+5, -3.85e+4, -3.85e+4, -3.3e+4,  1.48e+5,  7.14e+4],
[-3.3e+4, -3.85e+4, -3.85e+4, -1.1e+5,  7.14e+4,  1.48e+5]])

#### Matriz de Rigidez Global

In [11]:
# Inicialização da matriz de rigidez global
K_global = np.zeros((2 * num_nodes, 2 * num_nodes))

# Montagem da matriz de rigidez global
for element in elements:
    k_local = stiffness_matrix_triangle_linear(element, nodes, E, nu, t)
    for i in range(3):
        for j in range(3):
            for k in range(2):
                for l in range(2):
                    K_global[2 * (element[i] - 1) + k, 2 * (element[j] - 1) + l] += k_local[2 * i + k, 2 * j + l]

# Exibindo a matriz de rigidez global
print('Matriz de rigidez global:')
display(sp.Matrix(K_global).evalf(3))

Matriz de rigidez global:


Matrix([
[-1.48e+5, -7.14e+4, 3.85e+4,   3.3e+4,   1.1e+5, 3.85e+4,        0,        0],
[-7.14e+4, -1.48e+5, 3.85e+4,   1.1e+5,   3.3e+4, 3.85e+4,        0,        0],
[ 3.85e+4,  3.85e+4, 7.14e+4,        0,        0, -5.5e+3,  -1.1e+5,  -3.3e+4],
[  3.3e+4,   1.1e+5,       0, -7.14e+4,   5.5e+3,       0, -3.85e+4, -3.85e+4],
[  1.1e+5,   3.3e+4,       0,   5.5e+3, -7.14e+4,       0, -3.85e+4, -3.85e+4],
[ 3.85e+4,  3.85e+4, -5.5e+3,        0,        0, 7.14e+4,  -3.3e+4,  -1.1e+5],
[       0,        0, -1.1e+5, -3.85e+4, -3.85e+4, -3.3e+4,  1.48e+5,  7.14e+4],
[       0,        0, -3.3e+4, -3.85e+4, -3.85e+4, -1.1e+5,  7.14e+4,  1.48e+5]])

#### Vetor de Forças

O vetor de forças para cada nó devido a uma carga distribuída é calculado integrando a carga sobre o elemento:

$$
[F] = \int _A [S]^Tp dA
$$

onde:

$$
S = 
\begin{bmatrix}
S_i & 0 & S_j & 0 & S_k & 0 \\
0 & S_i & 0 & S_j & 0 & S_k\\
\end{bmatrix} \\
$$

$S_i = (1-\frac{x}{a})(1-\frac{y}{b})$

$S_j = \frac{x}{a}(1 - \frac{y}{b})$

$S_k = \frac{x}{a} \frac{y}{b}$

In [15]:
def calcula_vetor_forca(p, t):   
    F = np.zeros(6)
    F[0:3:2] = p * a * b / 2
    return F * t

F_global = calcula_vetor_forca(p, t)

print('Vetor de forças externas:')
display(sp.Matrix(F_global).evalf(3))

Vetor de forças externas:


Matrix([
[120.0],
[    0],
[120.0],
[    0],
[    0],
[    0]])

#### Aplicando Condições de Contorno

Conforme o enunciado, consideraremos os nós 3 e 4 como fixos.

In [19]:
# Aplicação das condições de contorno (nós 3 e 4 fixos)
fixed_nodes = [3, 4]
free_dof = np.setdiff1d(np.arange(2 * num_nodes), np.array([2 * (node - 1) for node in fixed_nodes] + [2 * node - 1 for node in fixed_nodes]))

# Redução da matriz de rigidez e do vetor de forças para os graus de liberdade livres
K_reduced = K_global[np.ix_(free_dof, free_dof)]

# Redução do vetor de forças
F_reduced = F_global[free_dof]

#### Deslocamentos


Resolvendo o sistema para determinar $u$, chegamos em:

In [20]:
# Resolvendo o sistema para obter os deslocamentos nos graus de liberdade livres
u_free = np.linalg.solve(K_reduced, F_reduced)

# Montagem do vetor de deslocamentos completo (incluindo zeros para graus de liberdade fixos)
u = np.zeros(2 * num_nodes)
u[free_dof] = u_free

print('Deslocamentos:')
for node in range(num_nodes):
    print('Nó {}: ({:.8f}, {:.8f})'.format(node + 1, u[2 * node], u[2 * node + 1]))


Deslocamentos:
Nó 1: (0.00156000, -0.00713143)
Nó 2: (0.00468000, -0.01025143)
Nó 3: (0.00000000, 0.00000000)
Nó 4: (0.00000000, 0.00000000)


#### Cálculo das Tensões



In [27]:
def stress_triangle_linear(element, nodes, u, E, nu):
    # Coordenadas dos nós do elemento
    x = nodes[element - 1, 0]
    y = nodes[element - 1, 1]

    # Área do elemento
    A = 0.5 * np.linalg.det(np.array([[1, x[0], y[0]],
                                      [1, x[1], y[1]],
                                      [1, x[2], y[2]]]))

    # Matriz B
    B = 1 / (2 * A) * np.array([
        [y[1] - y[2], 0, y[2] - y[0], 0, y[0] - y[1], 0],
        [0, x[2] - x[1], 0, x[0] - x[2], 0, x[1] - x[0]],
        [x[2] - x[1], y[1] - y[2], x[0] - x[2], y[2] - y[0], x[1] - x[0], y[0] - y[1]]
    ])

    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Vetor de deslocamentos do elemento
    u_element = np.array([u[2 * (node - 1):2 * node] for node in element]).flatten()

    # Cálculo das tensões
    stress = np.dot(D, np.dot(B, u_element))

    return stress

# Correção e cálculo das tensões
stresses = np.zeros((num_elements, 3))  # Inicializando o vetor de tensões

# Cálculo das tensões para cada elemento
for i, element in enumerate(elements):
    stresses[i] = stress_triangle_linear(element, nodes, u, E, nu)

print('Tensões nos elementos:')
for element in range(num_elements):
    print('Elemento {}: ({:.2f}, {:.2f}, {:.2f}) Pa'.format(element + 1, stresses[element, 0], stresses[element, 1], stresses[element, 2]))

Tensões nos elementos:
Elemento 1: (137.14, 197.14, -197.14) Pa
Elemento 2: (257.14, 77.14, -197.14) Pa


### b) Elemento Retangular



#### Funções de Forma
As funções de forma para elementos retangulares são bilineares e dependem das coordenadas dos nós.

#### Deformação e Tensão:

Para um elemento retangular em análise de elementos finitos, as funções de forma são tipicamente bilineares. Supondo um elemento retangular com quatro nós, as funções de forma podem ser expressas em termos das coordenadas locais $\xi$ e $\eta$, que variam de -1 a 1. Para cada nó $i$, a função de forma $N_i(\xi, \eta)$ é definida como:

- Para o nó 1 (canto inferior esquerdo): $N_1 = \frac{1}{4}(1-\xi)(1-\eta)$
- Para o nó 2 (canto inferior direito): $N_2 = \frac{1}{4}(1+\xi)(1-\eta)$
- Para o nó 3 (canto superior direito): $N_3 = \frac{1}{4}(1+\xi)(1+\eta)$
- Para o nó 4 (canto superior esquerdo): $N_4 = \frac{1}{4}(1-\xi)(1+\eta)$


A relação entre tensão e deformação em um estado plano de tensões é dada pela lei de Hooke. Para um material isotrópico linear elástico, a matriz de elasticidade $[D]$ é usada para relacionar a tensão $\sigma$ à deformação $\varepsilon$:

$$
[D] = \frac{E}{(1-\nu^2)}\begin{bmatrix} 1 & \nu & 0 \\ \nu & 1 & 0 \\ 0 & 0 & \frac{1-\nu}{2} \end{bmatrix}
$$

Onde $E$ é o módulo de elasticidade e $\nu$ é o coeficiente de Poisson do material. A tensão $\sigma$ é então calculada como $\sigma = [D]\varepsilon$.


A relação deformação-deslocamento é expressa como $\varepsilon = [B]\{d\}$, onde $\{d\}$ é o vetor dos deslocamentos nodais.


#### Cálculo das Matrizes de Rigidez

A formulação é similar: 
$$
[K] = \int_A [B]^T [D] [B] \, dA
$$

In [30]:
import numpy as np

# Propriedades do material
E = 200000  # Módulo de elasticidade (N/mm²)
nu = 0.3    # Coeficiente de Poisson
t = 1       # Espessura da viga (mm)

# Dimensões da viga para N = 1
a = b = 4   # Dimensões

# Carga aplicada
p = 150 * (1 + 1) / 20

# Coordenadas dos nós (em mm)
nodes = np.array([
    [0, 0],  # Nó 1
    [a, 0],  # Nó 2
    [a, b],  # Nó 3
    [0, b]   # Nó 4
])

# Número de nós
num_nodes = len(nodes)

# Função para calcular a matriz de rigidez de um elemento retangular
def stiffness_matrix_rectangular(E, nu, t, a, b):
    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Matriz de rigidez do elemento retangular
    k = t * a * b / 36 * np.array([
        [4, 2, -1, -2, -2, -1, -1, 0],
        [2, 4, 0, -1, -1, -2, -2, -1],
        [-1, 0, 4, 2, 1, 0, -2, -1],
        [-2, -1, 2, 4, 0, 1, -1, -2],
        [-2, -1, 1, 0, 4, 2, -1, 0],
        [-1, -2, 0, 1, 2, 4, 0, -1],
        [-1, -2, -2, -1, -1, 0, 4, 2],
        [0, -1, -1, -2, 0, -1, 2, 4]
    ])

    return k

# Aplicando a matriz de rigidez para um elemento retangular
K_rectangular = stiffness_matrix_rectangular(E, nu, t, a, b)
print('Matriz de rigidez para um elemento retangular:')
display(sp.Matrix(K_rectangular).evalf(3))

Matriz de rigidez para um elemento retangular:


Matrix([
[  1.78,  0.889, -0.444, -0.889, -0.889, -0.444, -0.444,      0],
[ 0.889,   1.78,      0, -0.444, -0.444, -0.889, -0.889, -0.444],
[-0.444,      0,   1.78,  0.889,  0.444,      0, -0.889, -0.444],
[-0.889, -0.444,  0.889,   1.78,      0,  0.444, -0.444, -0.889],
[-0.889, -0.444,  0.444,      0,   1.78,  0.889, -0.444,      0],
[-0.444, -0.889,      0,  0.444,  0.889,   1.78,      0, -0.444],
[-0.444, -0.889, -0.889, -0.444, -0.444,      0,   1.78,  0.889],
[     0, -0.444, -0.444, -0.889,      0, -0.444,  0.889,   1.78]])

#### Deslocamentos

In [31]:
# Aplicando as condições de contorno (nós 3 e 4 fixos)
fixed_nodes = [3, 4]
free_dof_rect = np.setdiff1d(np.arange(2 * num_nodes), np.array([2 * (node - 1) for node in fixed_nodes] + [2 * node - 1 for node in fixed_nodes]))

# Redução da matriz de rigidez e do vetor de forças para os graus de liberdade livres
K_reduced_rect = K_rectangular[np.ix_(free_dof_rect, free_dof_rect)]

# Vetor de forças externas (considerando a carga distribuída)
F_rect = np.zeros(2 * num_nodes)
# Aplicando a carga no nó 2 (direção y)
F_rect[2 * 2 - 1] = -p * a * b / 2  # Carga dividida igualmente entre os nós 2 e 3

# Redução do vetor de forças
F_reduced_rect = F_rect[free_dof_rect]

# Resolvendo o sistema para obter os deslocamentos nos graus de liberdade livres
u_free_rect = np.linalg.solve(K_reduced_rect, F_reduced_rect)

# Montagem do vetor de deslocamentos completo (incluindo zeros para graus de liberdade fixos)
u_rect = np.zeros(2 * num_nodes)
u_rect[free_dof_rect] = u_free_rect

# Deslocamentos nos nós para o elemento retangular
print('Deslocamentos no elemento retangular:')
for node in range(num_nodes):
    print('Nó {}: ({:.8f}, {:.8f})'.format(node + 1, u_rect[2 * node], u_rect[2 * node + 1]))

Deslocamentos no elemento retangular:
Nó 1: (-41.14285714, -7.71428571)
Nó 2: (46.28571429, -113.14285714)
Nó 3: (0.00000000, 0.00000000)
Nó 4: (0.00000000, 0.00000000)


#### Tensões

In [17]:
# Função para calcular as tensões em um elemento retangular
def stress_rectangular(E, nu, u, a, b):
    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Gradientes de deslocamento
    Bx = 1 / (2 * a) * np.array([-1, 1, 1, -1])
    By = 1 / (2 * b) * np.array([-1, -1, 1, 1])

    # Deslocamentos nos nós do elemento
    u_x = u[::2]
    u_y = u[1::2]

    # Cálculo das deformações
    epsilon_x = np.dot(Bx, u_x)
    epsilon_y = np.dot(By, u_y)
    gamma_xy = np.dot(Bx, u_y) + np.dot(By, u_x)

    # Cálculo das tensões
    stress = np.dot(D, np.array([epsilon_x, epsilon_y, gamma_xy]))

    return stress

# Calculando as tensões no elemento retangular
stress_rect = stress_rectangular(E, nu, u_rect, a, b)
print('Tensões no elemento retangular: ({:.3f}, {}, {})'.format(stress_rect[0], stress_rect[1], stress_rect[2]))


Tensões no elemento retangular: (3397959.18367347, 4040816.3265306125, -1063186.8131868134)
